In [1]:
import torch
torch.cuda.set_device(1)
print('cuda:', torch.cuda.current_device())
device = "cuda" if torch.cuda.is_available() else "cpu"

cuda: 1


# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
segment_len = 128
d_model = 160
conform_dropout = 0.1
sa_dim = 32
model_name = 'HW04-conform-sa32_dmodel160'

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker neme to their corresponding id.
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append(
                    [utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if self.segment_len == -1:
            mel = torch.FloatTensor(mel)
        elif len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start + self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
            
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(
        mel, batch_first=True,
        padding_value=-20)  # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir, segment_len)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
#         print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
        out = out.view(32, -1)      
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
    


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps))
        return max(
            0.0, 0.5 *
            (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [7]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset),
                ncols=0,
                desc="Valid",
                unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

# Main function

In [9]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "save_path": model_name,
        "batch_size": 32,
        "n_workers": 2,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 120000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(
        data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!", flush=True)

    model = Classifier(n_spks=speaker_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps,
                                                total_steps)
    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(
                f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})"
            )

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:19<00:00, 25.25 step/s, accuracy=0.50, loss=2.34, step=2000]
Valid: 100% 6944/6944 [00:02<00:00, 2584.85 uttr/s, accuracy=0.47, loss=2.48]
Train: 100% 2000/2000 [01:33<00:00, 21.41 step/s, accuracy=0.72, loss=1.28, step=4000]
Valid: 100% 6944/6944 [00:02<00:00, 2601.11 uttr/s, accuracy=0.62, loss=1.67]
Train: 100% 2000/2000 [01:31<00:00, 21.86 step/s, accuracy=0.69, loss=1.22, step=6000]
Valid: 100% 6944/6944 [00:02<00:00, 2674.94 uttr/s, accuracy=0.66, loss=1.47]
Train: 100% 2000/2000 [01:30<00:00, 22.05 step/s, accuracy=0.81, loss=0.85, step=8000]
Valid: 100% 6944/6944 [00:02<00:00, 2618.47 uttr/s, accuracy=0.71, loss=1.25]
Train: 100% 2000/2000 [01:33<00:00, 21.30 step/s, accuracy=0.88, loss=0.59, step=1e+4]
Valid: 100% 6944/6944 [00:02<00:00, 2625.68 uttr/s, accuracy=0.74, loss=1.12]
Train:   0% 4/2000 [00:00<01:59, 16.70 step/s, accuracy=0.75, loss=1.04, step=1e+4]

Step 10000, best model saved. (accuracy=0.7421)


Train: 100% 2000/2000 [01:31<00:00, 21.88 step/s, accuracy=0.81, loss=0.83, step=12000]
Valid: 100% 6944/6944 [00:02<00:00, 2736.30 uttr/s, accuracy=0.74, loss=1.10]
Train: 100% 2000/2000 [01:32<00:00, 21.57 step/s, accuracy=0.84, loss=0.53, step=14000]
Valid: 100% 6944/6944 [00:02<00:00, 2580.12 uttr/s, accuracy=0.76, loss=1.05]
Train: 100% 2000/2000 [01:31<00:00, 21.96 step/s, accuracy=0.91, loss=0.47, step=16000]
Valid: 100% 6944/6944 [00:02<00:00, 2685.89 uttr/s, accuracy=0.77, loss=1.02]
Train: 100% 2000/2000 [01:33<00:00, 21.41 step/s, accuracy=0.91, loss=0.25, step=18000]
Valid: 100% 6944/6944 [00:02<00:00, 2791.96 uttr/s, accuracy=0.78, loss=0.94]
Train: 100% 2000/2000 [01:32<00:00, 21.54 step/s, accuracy=0.88, loss=0.29, step=2e+4] 
Valid: 100% 6944/6944 [00:02<00:00, 2705.88 uttr/s, accuracy=0.78, loss=0.93]
Train:   0% 4/2000 [00:00<02:10, 15.25 step/s, accuracy=0.91, loss=0.32, step=2e+4]

Step 20000, best model saved. (accuracy=0.7810)


...

Step 100000, best model saved. (accuracy=0.9032)


Train: 100% 2000/2000 [01:30<00:00, 22.10 step/s, accuracy=1.00, loss=0.00, step=102000]
Valid: 100% 6944/6944 [00:02<00:00, 2626.69 uttr/s, accuracy=0.90, loss=0.48]
Train: 100% 2000/2000 [01:27<00:00, 22.98 step/s, accuracy=0.97, loss=0.12, step=104000]
Valid: 100% 6944/6944 [00:02<00:00, 2694.68 uttr/s, accuracy=0.90, loss=0.48]
Train: 100% 2000/2000 [01:27<00:00, 22.86 step/s, accuracy=1.00, loss=0.00, step=106000]
Valid: 100% 6944/6944 [00:02<00:00, 2796.66 uttr/s, accuracy=0.90, loss=0.46]
Train: 100% 2000/2000 [01:27<00:00, 22.93 step/s, accuracy=1.00, loss=0.00, step=108000]
Valid: 100% 6944/6944 [00:02<00:00, 2764.45 uttr/s, accuracy=0.90, loss=0.50]
Train: 100% 2000/2000 [01:28<00:00, 22.59 step/s, accuracy=1.00, loss=0.00, step=110000]
Valid: 100% 6944/6944 [00:02<00:00, 2676.55 uttr/s, accuracy=0.90, loss=0.50]
Train:   0% 4/2000 [00:00<01:51, 17.86 step/s, accuracy=1.00, loss=0.00, step=110004]

Step 110000, best model saved. (accuracy=0.9048)


Train: 100% 2000/2000 [01:28<00:00, 22.72 step/s, accuracy=1.00, loss=0.00, step=112000]
Valid: 100% 6944/6944 [00:02<00:00, 2664.65 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [01:27<00:00, 22.82 step/s, accuracy=1.00, loss=0.01, step=114000]
Valid: 100% 6944/6944 [00:02<00:00, 2747.62 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [01:28<00:00, 22.69 step/s, accuracy=1.00, loss=0.00, step=116000]
Valid: 100% 6944/6944 [00:02<00:00, 2571.79 uttr/s, accuracy=0.91, loss=0.47]
Train: 100% 2000/2000 [01:27<00:00, 22.81 step/s, accuracy=1.00, loss=0.01, step=118000]
Valid: 100% 6944/6944 [00:02<00:00, 2677.77 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:20<00:00, 24.96 step/s, accuracy=1.00, loss=0.01, step=120000]
Valid: 100% 6944/6944 [00:01<00:00, 4085.63 uttr/s, accuracy=0.91, loss=0.47]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 120000, best model saved. (accuracy=0.9088)


# Inference

## Dataset of inference

In [10]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(32, -1)  
        print(out.shape)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(32,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([32, 32, 150])
torch.Size([32, 5120])


torch.Size([32, 600])

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(1, -1)  
        print(out.shape)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(1,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([1, 32, 150])
torch.Size([1, 5120])


torch.Size([1, 600])

In [13]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "model_path": model_name,
        "output_path": "./output_{}.csv".format(model_name),
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=8,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!", flush=True)

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(n_spks=speaker_num).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"[Info]: Finish creating model!", flush=True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            print(mels.shape)
            outs = model(mels)
            print(outs.shape)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/6000 [00:00<?, ?it/s]

torch.Size([1, 2112, 40])
A torch.Size([1, 32, 2112])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 586, 40])
A torch.Size([1, 32, 586])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 465, 40])
A torch.Size([1, 32, 465])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 32, 404])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 402, 40])
A torch.Size([1, 32, 402])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 428, 40])
A torch.Size([1, 32, 428])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 899, 40])
A torch.Size([1, 32, 899])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1677, 40])
A torch.Size([1, 32, 1677])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 335, 40])
A torch.Size([1, 32, 335])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 528, 40])
A torch.Size([1, 32, 528])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 533, 40])
A torch.Size([1, 32, 

torch.Size([1, 2048, 40])
A torch.Size([1, 32, 2048])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 729, 40])
A torch.Size([1, 32, 729])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1133, 40])
A torch.Size([1, 32, 1133])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 424, 40])
A torch.Size([1, 32, 424])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 361, 40])
A torch.Size([1, 32, 361])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 616, 40])
A torch.Size([1, 32, 616])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 594, 40])
A torch.Size([1, 32, 594])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 890, 40])
A torch.Size([1, 32, 890])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 400, 40])
A torch.Size([1, 32, 400])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 32, 404])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 556, 40])
A torch.Size([1, 32, 

A torch.Size([1, 32, 2095])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 309, 40])
A torch.Size([1, 32, 309])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 472, 40])
A torch.Size([1, 32, 472])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 370, 40])
A torch.Size([1, 32, 370])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 295, 40])
A torch.Size([1, 32, 295])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 528, 40])
A torch.Size([1, 32, 528])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 883, 40])
A torch.Size([1, 32, 883])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1089, 40])
A torch.Size([1, 32, 1089])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 961, 40])
A torch.Size([1, 32, 961])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 427, 40])
A torch.Size([1, 32, 427])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1264, 40])
A torch.Size([1, 32, 1264])
torch.Size([1, 512

...

torch.Size([1, 540, 40])
A torch.Size([1, 32, 540])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1200, 40])
A torch.Size([1, 32, 1200])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 586, 40])
A torch.Size([1, 32, 586])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 388, 40])
A torch.Size([1, 32, 388])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 377, 40])
A torch.Size([1, 32, 377])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 504, 40])
A torch.Size([1, 32, 504])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 516, 40])
A torch.Size([1, 32, 516])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1008, 40])
A torch.Size([1, 32, 1008])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 411, 40])
A torch.Size([1, 32, 411])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 537, 40])
A torch.Size([1, 32, 537])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1472, 40])
A torch.Size([1, 32,

A torch.Size([1, 32, 634])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 722, 40])
A torch.Size([1, 32, 722])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 507, 40])
A torch.Size([1, 32, 507])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 537, 40])
A torch.Size([1, 32, 537])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 416, 40])
A torch.Size([1, 32, 416])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 392, 40])
A torch.Size([1, 32, 392])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 563, 40])
A torch.Size([1, 32, 563])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 680, 40])
A torch.Size([1, 32, 680])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 449, 40])
A torch.Size([1, 32, 449])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 772, 40])
A torch.Size([1, 32, 772])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 568, 40])
A torch.Size([1, 32, 568])
torch.Size([1, 5120])
t

torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 876, 40])
A torch.Size([1, 32, 876])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1260, 40])
A torch.Size([1, 32, 1260])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 601, 40])
A torch.Size([1, 32, 601])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 552, 40])
A torch.Size([1, 32, 552])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1024, 40])
A torch.Size([1, 32, 1024])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1819, 40])
A torch.Size([1, 32, 1819])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1635, 40])
A torch.Size([1, 32, 1635])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 680, 40])
A torch.Size([1, 32, 680])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 498, 40])
A torch.Size([1, 32, 498])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 923, 40])
A torch.Size([1, 32, 923])
torch.Size([1, 5120])
torch.Size([1, 600])

torch.Size([1, 2733, 40])
A torch.Size([1, 32, 2733])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 390, 40])
A torch.Size([1, 32, 390])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1640, 40])
A torch.Size([1, 32, 1640])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1664, 40])
A torch.Size([1, 32, 1664])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 690, 40])
A torch.Size([1, 32, 690])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 430, 40])
A torch.Size([1, 32, 430])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 426, 40])
A torch.Size([1, 32, 426])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 892, 40])
A torch.Size([1, 32, 892])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 397, 40])
A torch.Size([1, 32, 397])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 32, 404])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 449, 40])
A torch.Size([1, 32

A torch.Size([1, 32, 604])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 650, 40])
A torch.Size([1, 32, 650])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 629, 40])
A torch.Size([1, 32, 629])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1495, 40])
A torch.Size([1, 32, 1495])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1098, 40])
A torch.Size([1, 32, 1098])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 913, 40])
A torch.Size([1, 32, 913])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 965, 40])
A torch.Size([1, 32, 965])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1444, 40])
A torch.Size([1, 32, 1444])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 689, 40])
A torch.Size([1, 32, 689])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 345, 40])
A torch.Size([1, 32, 345])
torch.Size([1, 5120])
torch.Size([1, 600])
torch.Size([1, 1506, 40])
A torch.Size([1, 32, 1506])
torch.Size([1, 